In [2]:
import matplotlib.pyplot as plt
import librosa 
import os
import numpy as np
import pandas as pd
import timeit
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import  MinMaxScaler
import heapq
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import datasets, model_selection
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

'''
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model = svm.svc(kernel='linear', c=1, gamma=1) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model.fit(X, y)
model.score(X, y)
'''
import warnings
warnings.filterwarnings("ignore")




scaler=MinMaxScaler()
tr_f=os.listdir('train/Train/')
#tr_f=os.listdir('train/Train/')

#print(tr_f[0])

print('Number of clips used for training are:{}'.format(int(len(tr_f)*0.85)))
#tr_f.sort()
fstart=timeit.default_timer()

#print('\nShowing First 5 of training set:{}'.format(tr_f[:5]))
data=pd.read_csv('train/train.csv')
#data=pd.read_csv('train/train.csv')

print(data)

y=list(data['Class'].values)
#y=np.array(y)
#y=y.reshape(len(tr_f),1)




types=list(data.Class.unique())
print('Guessing from classes : {}'.format(types))





Number of clips used for training are:4619
        ID             Class
0        0             siren
1        1      street_music
2        2          drilling
3        3             siren
4        4          dog_bark
5        6  children_playing
6       10      street_music
7       11          drilling
8       12          gun_shot
9       15          dog_bark
10      17     engine_idling
11      18             siren
12      19          gun_shot
13      20     engine_idling
14      22   air_conditioner
15      24   air_conditioner
16      26     engine_idling
17      27             siren
18      32  children_playing
19      33        jackhammer
20      35          gun_shot
21      36             siren
22      37        jackhammer
23      38      street_music
24      40        jackhammer
25      42     engine_idling
26      43             siren
27      44  children_playing
28      45        jackhammer
29      46     engine_idling
...    ...               ...
5405  8691      street_music


In [37]:
x,sr=librosa.load('train/Train/'+tr_f[0])
mfccs=librosa.feature.mfcc(x,sr=sr)
print('Sampling rate : ',sr,'\nTotal frames considered (4*sr/512) : ',int(4*sr/512))
print('Shape of mfccs : ',np.shape(mfccs))

Sampling rate :  22050 
Total frames considered (4*sr/512) :  172
Shape of mfccs :  (20, 173)


In [3]:
count=0

for file in tr_f:
	avg=[]
	x,sr=librosa.load('train/Train/'+file)
#	x,sr=librosa.load('train/Train/'+file)

	#print(file,sr,np.shape(x))
	mfccs=librosa.feature.mfcc(x,sr=sr)
	#print(len(mfccs))
	if(count==0):
		a=[]
		for i in range(len(mfccs)):
			a.append(i)	
			b=pd.DataFrame(columns=a)
	#print(np.shape(mfccs))
	#print(np.shape(sum(mfccs)),len(sum(mfccs)))
	#print(mfccs[0],np.shape(mfccs[0]))	
	for i in range(len(mfccs)):
		avgapp=sum(mfccs[i])/len(mfccs[i])
		avg.append(avgapp)
		varapp=sum(np.power(mfccs[i]-avgapp,2))/len(mfccs[i])
		#avg.append(varapp)
#	print(mfccs,np.shape(mfccs),type(mfccs))
	avg=np.array(avg).reshape(1,len(avg))	
	#print(mfccs,np.shape(mfccs),type(mfccs))
	avg=pd.DataFrame(data=avg[0:],index=[int(file.replace('.wav',''))])	
	b=b.append(avg)
	count=count+1
	ll=int(len(tr_f)/6)	
	if(count%ll==0):
#	if(count%300==0):
		stop=timeit.default_timer()
		print('\nReached: {}'.format(count))
		print('time :{:.2f}'.format(stop-fstart))
print('Time taken for feature extraction: {:.2f}'.format(timeit.default_timer()-fstart))	
b=b.sort_index(axis=0)
p=pd.DataFrame(data=y,index=list(data['ID'].values),columns=['label'])
b=b.fillna(0)
b= pd.concat([b, p], axis=1)
b[a]=scaler.fit_transform(b[a])

X=b[a]



Reached: 905
time :195.62

Reached: 1810
time :387.90

Reached: 2715
time :582.37

Reached: 3620
time :781.46

Reached: 4525
time :1004.30

Reached: 5430
time :1219.63
Time taken for feature extraction: 1220.83


In [4]:
b_tr,b_ts,y_tr,y_ts = train_test_split(X,y,test_size=0.15,random_state=22)

In [5]:
b_tr,b_ts,y_tr,y_ts = train_test_split(X,y,test_size=0.15,random_state=22)
k=int(np.power(np.array(list((b_tr).shape)),0.5)[0])
if(k%2==0):
	k=k+1

Rstart=timeit.default_timer()
scorek=0
for i in range(1,k+1):
	for j in range(2,3):

		knn = KNeighborsClassifier(n_neighbors = i,p=j)
		knn.fit(b_tr,y_tr)
		if(scorek < knn.score(b_ts,y_ts)):
			scorek = knn.score(b_ts,y_ts)
			finalk=[i,j]	
Rstop=timeit.default_timer()
print('Time taken for knn : {:.2f}'.format(Rstop-Rstart),'Score of knn: {:.2f}'.format(scorek),'[K,p] used ={}'.format(finalk))
knn = KNeighborsClassifier(n_neighbors = finalk[0],p=finalk[1])
knn.fit(b_tr,y_tr)
print('Classification Report for knn :- \n',classification_report(y_ts,knn.predict(b_ts)))


Time taken for knn : 9.74 Score of knn: 0.95 [K,p] used =[1, 2]
Classification Report for knn :- 
                   precision    recall  f1-score   support

 air_conditioner       0.97      0.99      0.98        87
        car_horn       0.98      0.88      0.93        50
children_playing       0.94      0.93      0.93        95
        dog_bark       0.93      0.87      0.90        93
        drilling       0.99      0.96      0.97        81
   engine_idling       0.97      1.00      0.98        88
        gun_shot       0.84      0.94      0.89        34
      jackhammer       0.97      1.00      0.98       113
           siren       0.98      0.99      0.98        87
    street_music       0.91      0.91      0.91        88

        accuracy                           0.95       816
       macro avg       0.95      0.95      0.95       816
    weighted avg       0.95      0.95      0.95       816



In [6]:
X1=X.drop([18],axis=1)
b_tr,b_ts,y_tr,y_ts = train_test_split(X1,y,test_size=0.15,random_state=22)
k=int(np.power(np.array(list((b_tr).shape)),0.5)[0])
if(k%2==0):
	k=k+1

Rstart=timeit.default_timer()
scorek=0
for i in range(1,k+1):
	for j in range(2,3):

		knn = KNeighborsClassifier(n_neighbors = i,p=j)
		knn.fit(b_tr,y_tr)
		if(scorek < knn.score(b_ts,y_ts)):
			scorek = knn.score(b_ts,y_ts)
			finalk=[i,j]	
Rstop=timeit.default_timer()
print('Time taken for knn : {:.2f}'.format(Rstop-Rstart),'Score of knn: {:.2f}'.format(scorek),'[K,p] used ={}'.format(finalk))
knn = KNeighborsClassifier(n_neighbors = finalk[0],p=finalk[1])
knn.fit(b_tr,y_tr)
print('Classification Report for knn :- \n',classification_report(y_ts,knn.predict(b_ts)))


Time taken for knn : 8.95 Score of knn: 0.96 [K,p] used =[1, 2]
Classification Report for knn :- 
                   precision    recall  f1-score   support

 air_conditioner       0.98      0.99      0.98        87
        car_horn       0.98      0.86      0.91        50
children_playing       0.96      0.96      0.96        95
        dog_bark       0.92      0.87      0.90        93
        drilling       0.97      0.96      0.97        81
   engine_idling       0.98      1.00      0.99        88
        gun_shot       0.84      0.94      0.89        34
      jackhammer       0.97      1.00      0.98       113
           siren       0.99      0.99      0.99        87
    street_music       0.93      0.94      0.94        88

        accuracy                           0.96       816
       macro avg       0.95      0.95      0.95       816
    weighted avg       0.96      0.96      0.96       816



In [18]:
x=list(range(1,100))
y=[]
y1=[]

clf=RandomForestClassifier()
clf.fit(b_tr,y_tr)
print(clf.score(b_ts,y_ts))
print(clf.score(b_tr,y_tr))    
#print(max(y),x[y.index(max(y))])
#print(max(y1),x[y1.index(max(y1))])
#plt.plot(x,y1,linewidth=2.0)
#plt.plot(x,y,linewidth=2.0)
#plt.show()

0.8419117647058824
0.9982680233816843


In [56]:
y=list(data['Class'].values)

b_tr,b_ts,y_tr,y_ts = train_test_split(X1,y,test_size=0.15,random_state=22)

In [57]:

k=int(np.power(np.array(list((b_tr).shape)),0.5)[0])
if(k%2==0):
	k=k+1

Rstart=timeit.default_timer()
scorek=0
for i in range(1,k+1):
	for j in range(2,3):

		knn = KNeighborsClassifier(n_neighbors = i,p=j)
		knn.fit(b_tr,y_tr)
		if(scorek < knn.score(b_ts,y_ts)):
			scorek = knn.score(b_ts,y_ts)
			finalk=[i,j]	
Rstop=timeit.default_timer()
print('Time taken for knn : {:.2f}'.format(Rstop-Rstart),'Score of knn: {:.2f}'.format(scorek),'[K,p] used ={}'.format(finalk))

#print('Classification Report for knn :- \n',classification_report(y_ts,knn.predict(b_ts)))


Time taken for knn : 10.54 Score of knn: 0.96 [K,p] used =[1, 2]


In [1]:
li=list(range(0,21))
y1=[]
y2=[]
X1=X
for i in li:
    if(i!=20):
        X1=X.drop([i],axis=1)
    b_tr,b_ts,y_tr,y_ts = train_test_split(X1,y,test_size=0.15,random_state=22)
    clf=RandomForestClassifier(n_estimators=61)
    clf.fit(b_tr,y_tr)
    y2.append(clf.score(b_tr,y_tr))              
    y1.append(clf.score(b_ts,y_ts))
print(max(y2),li[y2.index(max(y2))])
print(max(y1),li[y1.index(max(y1))])
plt.plot(li,y1,linewidth=2.0)
plt.plot(li,y2,linewidth=2.0)
plt.show()              

NameError: name 'X' is not defined